In [2]:
pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 28.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 36.8 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
df = pd.read_csv("Customer-Churn.csv")
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


In [ ]:
# Dropping rows with missing values
df.dropna(inplace=True)

In [ ]:
# Separate features and label
x = df.drop(columns=['Churn']) # Feature columns
y = df[['Churn']] # Label

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.7, random_state = 620)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(4922, 19) (4922, 1)
(2110, 19) (2110, 1)


In [ ]:
# Ensure all data is stored as a DataFrame
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)
x_test = pd.DataFrame(x_test)
y_test = pd.DataFrame(y_test)

# Preprocessor for feature matrix x
x_scaler = StandardScaler()
x_encoder = OneHotEncoder(drop = 'first')
preprocessor = ColumnTransformer([('scaler', x_scaler, make_column_selector(dtype_include = 'number')),
                                  ('encoder', x_encoder, make_column_selector(dtype_include = 'object'))])

# Preprocessor for y
y_encoder = OneHotEncoder(drop = 'first')

# Transform data
x_train = preprocessor.fit_transform(x_train)
y_train = y_encoder.fit_transform(y_train).toarray().reshape(y_train.shape[0])

x_test = preprocessor.transform(x_test)
y_test = y_encoder.transform(y_test).toarray().reshape(y_test.shape[0])

In [ ]:
# Iterate through the groups of models to find the model with the highest accuracy
# using 5-fold cross-validation
for model in [LogisticRegression(), GaussianNB(), KNeighborsClassifier()]:
  cv_score = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=5)
  print("Accuracy score for %s: %.3f"%(model.__class__.__name__, cv_score.mean()))

Accuracy score for LogisticRegression: 0.802
Accuracy score for GaussianNB: 0.663
Accuracy score for KNeighborsClassifier: 0.746


In [ ]:
# Setting possible values for the penalty parameter of LogisticRegression()
lr_param_grid = {'penalty': ['l1', 'l2', 'none']}

# Create GridSearchCV for the model and pass in the parameter grid
log_reg = GridSearchCV(LogisticRegression(), lr_param_grid, cv=5,
                       scoring='accuracy', return_train_score=True,
                       n_jobs=-1, verbose=1)

# Fit the training data into the GridSearchCV
# This will determine which parameter returns the best model using cross validation
log_reg.fit(x_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solver l

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'none']},
             return_train_score=True, scoring='accuracy', verbose=1)

In [ ]:
# Get the best hyperparameters for the model
print(log_reg.best_params_)

# Get the best models from GridSearch
lg_model = log_reg.best_estimator_

lg_model.fit(x_train, y_train)

{'penalty': 'l2'}


LogisticRegression()

In [ ]:
# Make predictions on test data
y_predicted = lg_model.predict(x_test)

In [ ]:
print(accuracy_score(y_true=y_test, y_pred=y_predicted), '\n')

print(confusion_matrix(y_true=y_test, y_pred=y_predicted), '\n')

print(classification_report(y_test, y_predicted))

0.7834123222748816 

[[1507   57]
 [ 400  146]] 

              precision    recall  f1-score   support

         0.0       0.79      0.96      0.87      1564
         1.0       0.72      0.27      0.39       546

    accuracy                           0.78      2110
   macro avg       0.75      0.62      0.63      2110
weighted avg       0.77      0.78      0.74      2110

